# Fully connected feedforward network implementing a loss mask

In [ ]:
import torchmetrics.classification
from torch import cuda
assert cuda.is_available()
assert cuda.device_count() > 0

In [ ]:
print(cuda.get_device_name(cuda.current_device()))


NVIDIA GeForce RTX 3060 Ti


In [ ]:
import torch
import torch.nn as nn

In [ ]:
DEVICE = torch.device("cuda")
SEED = 76436278

torch.manual_seed(SEED)

### Load the Dataset

In [ ]:
from src.maldi2resistance.data.driams import Driams

driams = Driams(
    root_dir="/home/jan/Uni/master/data/Driams",
)

driams.loading_type = "memory"

driams

Loading Spectra into Memory:   0%|          | 0/55780 [00:00<?, ?it/s]

Antibiotic:,Amikacin,Amoxicillin-Clavulanic acid,Ampicillin,Ampicillin-Amoxicillin,Aztreonam,Benzylpenicillin,Cefazolin,Cefepime,Cefpodoxime,Ceftazidime,Ceftriaxone,Cefuroxime,Ciprofloxacin,Clarithromycin,Clindamycin,Colistin,Cotrimoxazole,Ertapenem,Erythromycin,Fosfomycin,Fosfomycin-Trometamol,Fusidic acid,Gentamicin,Imipenem,Levofloxacin,Meropenem,Mupirocin,Nitrofurantoin,Norfloxacin,Oxacillin,Penicillin,Piperacillin-Tazobactam,Polymyxin B,Rifampicin,Teicoplanin,Tetracycline,Tobramycin,Vancomycin
Number resistant:,1068,13366,8578,21966,628,618,4223,7383,2338,3470,8659,5855,9338,310,4381,2874,7405,427,5468,2303,1326,3620,3481,7560,4217,5194,570,1271,1205,5537,12431,7616,486,580,244,3534,1707,227
Number susceptible:,20941,24992,4194,4905,456,457,5813,31567,4382,24566,28464,8368,36822,1262,9841,15784,24590,21740,9044,10184,4803,8498,22662,31717,17989,27228,4656,3603,7031,7740,4286,31308,2305,14964,8486,10376,16809,20540
Number data points:,22009,38358,12772,26871,1084,1075,10036,38950,6720,28036,37123,14223,46160,1572,14222,18658,31995,22167,14512,12487,6129,12118,26143,39277,22206,32422,5226,4874,8236,13277,16717,38924,2791,15544,8730,13910,18516,20767


In [ ]:
len(driams.label_stats.columns)

38

In [ ]:
import copy
from maldi2resistance.model.MultilabelResMLP import MultilabelResMLP

model = MultilabelResMLP( input_dim= 18000, output_dim= len(driams.selected_antibiotics), hidden_dim=128)
model = model.to(DEVICE)

model_state = copy.deepcopy(model.state_dict()) 

In [ ]:
from torchinfo import summary

print(summary(model))

Layer (type:depth-idx)                        Param #
MultilabelResMLP                              --
├─Linear: 1-1                                 2,304,128
├─ResMLP: 1-2                                 --
│    └─Sequential: 2-1                        --
│    │    └─ResBlock: 3-1                     16,768
│    │    └─ResBlock: 3-2                     16,768
│    │    └─ResBlock: 3-3                     16,768
│    │    └─ResBlock: 3-4                     16,768
│    │    └─ResBlock: 3-5                     16,768
│    │    └─Linear: 3-6                       4,902
Total params: 2,392,870
Trainable params: 2,392,870
Non-trainable params: 0


In [ ]:
from torch.utils.data import DataLoader

In [ ]:
from maldi2resistance.metric.PrecisionRecall import MultiLabelPRNan
from pathlib import Path
from maldi2resistance.metric.ROC import MultiLabelRocNan
from maldi2resistance.loss.maskedLoss import MaskedBCE
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

print("Start training ...")
model.train()

batch_size = 128
fig_path = Path("./kfold/figures")
fig_path.mkdir(parents=True, exist_ok=True)

loss_per_batch = []

class_weights_negative = torch.tensor((1 - (driams.label_stats.loc["negative"] / driams.label_stats.loc["n_sum"])).values, device=DEVICE)
class_weights_positive = torch.tensor((1 - (driams.label_stats.loc["positive"] / driams.label_stats.loc["n_sum"])).values, device=DEVICE)

criterion = MaskedBCE(class_weights_positive= class_weights_negative, class_weights_negative= class_weights_negative)

gen = torch.Generator()

for fold, (train_data, test_data) in enumerate(driams.getK_fold(n_splits=5, shuffle=True, random_state= SEED)):
    
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, drop_last=True, generator= gen.manual_seed(SEED), pin_memory=True)
    test_loader = DataLoader(test_data, batch_size=len(test_data), shuffle=True, drop_last=True, generator= gen.manual_seed(SEED))
    
    model.load_state_dict(model_state)
    model.train()
    
    optimizer = Adam(model.parameters(), lr=1e-3, amsgrad = True)
    scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

    for epoch in tqdm(range(30)):
        overall_loss = 0
        
        for batch_idx, (x, y) in enumerate(train_loader):
    
            x = x.to(DEVICE, non_blocking=True)
            y = y.to(DEVICE, non_blocking=True)
            
            optimizer.zero_grad()
    
            output = model(x)
        
            loss = criterion(output, y)
            current_loss_value = loss.item()
            loss_per_batch.append(current_loss_value)
            
            overall_loss += current_loss_value
            
            loss.backward()
            optimizer.step()
    
        scheduler.step()
        with tqdm.external_write_mode():
            print(f"\tAverage Loss: {overall_loss / (batch_idx*batch_size):.6f} \tLearning rate: {scheduler.get_last_lr()[0]:.6f}")
    
    
    print(f"Finished Fold {fold}")
    
    model.eval()
    
    DEVICE = torch.device("cpu")
    test_features, test_labels = next(iter(test_loader))
    test_features = test_features.to(DEVICE)
    test_labels = test_labels.to(DEVICE)
    model = model.to(DEVICE)
    
    ml_roc = MultiLabelRocNan()
    output = model(test_features)
    
    
    ml_roc.compute(output,test_labels,driams.selected_antibiotics, create_csv=f"./kfold/csv/fold-{fold}_ROC.csv")
    fig_, ax_ = ml_roc()
    
    plt.savefig(fig_path / f"fold-{fold}_ROC.png", transparent=True, format= "png", bbox_inches = "tight")
    plt.close()
    
    ml_pr = MultiLabelPRNan()
    ml_pr.compute(output,test_labels,driams.selected_antibiotics, create_csv=f"./kfold/csv/fold-{fold}_PrecisionRecall.csv")
    
    fig_, ax_ = ml_pr()
    
    plt.savefig(fig_path / f"fold-{fold}_PrecisionRecall.png", transparent=True, format= "png", bbox_inches = "tight")
    plt.close()
    
    DEVICE = torch.device("cuda")
    model = model.to(DEVICE)

Start training ...


  0%|          | 0/30 [00:00<?, ?it/s]

	Average Loss: 0.000273 	Learning rate: 0.001000
	Average Loss: 0.000218 	Learning rate: 0.001000
	Average Loss: 0.000204 	Learning rate: 0.001000
	Average Loss: 0.000195 	Learning rate: 0.001000
	Average Loss: 0.000187 	Learning rate: 0.001000
	Average Loss: 0.000181 	Learning rate: 0.001000
	Average Loss: 0.000174 	Learning rate: 0.001000
	Average Loss: 0.000168 	Learning rate: 0.001000
	Average Loss: 0.000163 	Learning rate: 0.001000
	Average Loss: 0.000159 	Learning rate: 0.000500
	Average Loss: 0.000146 	Learning rate: 0.000500
	Average Loss: 0.000141 	Learning rate: 0.000500
	Average Loss: 0.000137 	Learning rate: 0.000500
	Average Loss: 0.000133 	Learning rate: 0.000500
	Average Loss: 0.000131 	Learning rate: 0.000500
	Average Loss: 0.000128 	Learning rate: 0.000500
	Average Loss: 0.000125 	Learning rate: 0.000500
	Average Loss: 0.000122 	Learning rate: 0.000500
	Average Loss: 0.000119 	Learning rate: 0.000500
	Average Loss: 0.000117 	Learning rate: 0.000250
	Average Loss: 0.000

  0%|          | 0/30 [00:00<?, ?it/s]

	Average Loss: 0.000270 	Learning rate: 0.001000
	Average Loss: 0.000218 	Learning rate: 0.001000
	Average Loss: 0.000205 	Learning rate: 0.001000
	Average Loss: 0.000195 	Learning rate: 0.001000
	Average Loss: 0.000188 	Learning rate: 0.001000
	Average Loss: 0.000182 	Learning rate: 0.001000
	Average Loss: 0.000176 	Learning rate: 0.001000
	Average Loss: 0.000170 	Learning rate: 0.001000
	Average Loss: 0.000165 	Learning rate: 0.001000
	Average Loss: 0.000161 	Learning rate: 0.000500
	Average Loss: 0.000147 	Learning rate: 0.000500
	Average Loss: 0.000143 	Learning rate: 0.000500
	Average Loss: 0.000140 	Learning rate: 0.000500
	Average Loss: 0.000136 	Learning rate: 0.000500
	Average Loss: 0.000134 	Learning rate: 0.000500
	Average Loss: 0.000131 	Learning rate: 0.000500
	Average Loss: 0.000128 	Learning rate: 0.000500
	Average Loss: 0.000126 	Learning rate: 0.000500
	Average Loss: 0.000123 	Learning rate: 0.000500
	Average Loss: 0.000121 	Learning rate: 0.000250
	Average Loss: 0.000

...

	Average Loss: 0.000273 	Learning rate: 0.001000
	Average Loss: 0.000219 	Learning rate: 0.001000
	Average Loss: 0.000205 	Learning rate: 0.001000
	Average Loss: 0.000197 	Learning rate: 0.001000
	Average Loss: 0.000188 	Learning rate: 0.001000
	Average Loss: 0.000183 	Learning rate: 0.001000
	Average Loss: 0.000178 	Learning rate: 0.001000
	Average Loss: 0.000173 	Learning rate: 0.001000
	Average Loss: 0.000166 	Learning rate: 0.001000
	Average Loss: 0.000161 	Learning rate: 0.000500
	Average Loss: 0.000147 	Learning rate: 0.000500
	Average Loss: 0.000143 	Learning rate: 0.000500
	Average Loss: 0.000139 	Learning rate: 0.000500
	Average Loss: 0.000137 	Learning rate: 0.000500
	Average Loss: 0.000134 	Learning rate: 0.000500
	Average Loss: 0.000130 	Learning rate: 0.000500
	Average Loss: 0.000129 	Learning rate: 0.000500
	Average Loss: 0.000125 	Learning rate: 0.000500
	Average Loss: 0.000123 	Learning rate: 0.000500
	Average Loss: 0.000122 	Learning rate: 0.000250
	Average Loss: 0.000

  0%|          | 0/30 [00:00<?, ?it/s]

	Average Loss: 0.000271 	Learning rate: 0.001000
	Average Loss: 0.000219 	Learning rate: 0.001000
	Average Loss: 0.000205 	Learning rate: 0.001000
	Average Loss: 0.000195 	Learning rate: 0.001000
	Average Loss: 0.000187 	Learning rate: 0.001000
	Average Loss: 0.000180 	Learning rate: 0.001000
	Average Loss: 0.000175 	Learning rate: 0.001000
	Average Loss: 0.000169 	Learning rate: 0.001000
	Average Loss: 0.000163 	Learning rate: 0.001000
	Average Loss: 0.000158 	Learning rate: 0.000500
	Average Loss: 0.000145 	Learning rate: 0.000500
	Average Loss: 0.000140 	Learning rate: 0.000500
	Average Loss: 0.000138 	Learning rate: 0.000500
	Average Loss: 0.000134 	Learning rate: 0.000500
	Average Loss: 0.000131 	Learning rate: 0.000500
	Average Loss: 0.000128 	Learning rate: 0.000500
	Average Loss: 0.000126 	Learning rate: 0.000500
	Average Loss: 0.000124 	Learning rate: 0.000500
	Average Loss: 0.000121 	Learning rate: 0.000500
	Average Loss: 0.000119 	Learning rate: 0.000250
	Average Loss: 0.000

  0%|          | 0/30 [00:00<?, ?it/s]

	Average Loss: 0.000273 	Learning rate: 0.001000
	Average Loss: 0.000219 	Learning rate: 0.001000
	Average Loss: 0.000207 	Learning rate: 0.001000
	Average Loss: 0.000197 	Learning rate: 0.001000
	Average Loss: 0.000189 	Learning rate: 0.001000
	Average Loss: 0.000183 	Learning rate: 0.001000
	Average Loss: 0.000176 	Learning rate: 0.001000
	Average Loss: 0.000170 	Learning rate: 0.001000
	Average Loss: 0.000164 	Learning rate: 0.001000
	Average Loss: 0.000160 	Learning rate: 0.000500
	Average Loss: 0.000146 	Learning rate: 0.000500
	Average Loss: 0.000141 	Learning rate: 0.000500
	Average Loss: 0.000138 	Learning rate: 0.000500
	Average Loss: 0.000134 	Learning rate: 0.000500
	Average Loss: 0.000131 	Learning rate: 0.000500
	Average Loss: 0.000130 	Learning rate: 0.000500
	Average Loss: 0.000127 	Learning rate: 0.000500
	Average Loss: 0.000124 	Learning rate: 0.000500
	Average Loss: 0.000123 	Learning rate: 0.000500
	Average Loss: 0.000119 	Learning rate: 0.000250
	Average Loss: 0.000

In [ ]:
import pandas
micro = []
macro = []

for fold in range(0,5):
    csv = pandas.read_csv(f"./kfold/csv/fold-{fold}_ROC.csv")
    micro.append(csv[csv["class"] == "micro"]["ROCAUC"])
    macro.append(csv[csv["class"] == "macro"]["ROCAUC"])

In [ ]:
import numpy as np

print(f"Mean\t: {np.mean(micro)}")
print(f" SD \t: {np.std(micro) }")

Mean	: 0.9173856735229492
 SD 	: 0.002511751948371241


In [ ]:
print(f"Mean\t: {np.mean(macro)}")
print(f" SD \t: {np.std(macro) }")

Mean	: 0.8863620356509561
 SD 	: 0.0023034541081740773


In [ ]:
import pandas
micro = []
macro = []

for fold in range(0,5):
    csv = pandas.read_csv(f"./kfold/csv/fold-{fold}_PrecisionRecall.csv")
    micro.append(csv[csv["class"] == "micro"]["PrecisionRecallAUC"])
    macro.append(csv[csv["class"] == "macro"]["PrecisionRecallAUC"])

In [ ]:
import numpy as np

print(f"Mean\t: {np.mean(micro)}")
print(f" SD \t: {np.std(micro) }")

Mean	: 0.8422750115394593
 SD 	: 0.003494611661618225


In [ ]:
print(f"Mean\t: {np.mean(macro)}")
print(f" SD \t: {np.std(macro) }")

Mean	: 0.718270679445643
 SD 	: 0.005746222799864039
